In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.ndimage

from skimage.feature import hog
#from skimage import data, color, exposure
from sklearn.model_selection import  train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.externals import joblib
from characters import characters
from sklearn.decomposition import PCA
from __future__ import print_function
from skimage.feature import hog
from numpy import linalg as LA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
np.set_printoptions(threshold=np.inf)


#from PIL import Image

%matplotlib inline

In [2]:
############################# Pre-processing data ##############################
"""output
x_train=(10000,1024)
y_train=(10000,1)
x_test=(2000,1024)
y_test=(2000,1)
type of data ='numpy.ndarray'
"""
verbose = True

np.random.seed(1)

def load_data():
    if verbose:
        print('Loading dataset...')
    m = 92000
    data = np.load('dataset/dataset.npz')

    x_train = data['arr_0']
    y_train = data['arr_1'].reshape(78200, 1)
    x_test = data['arr_2']
    y_test = data['arr_3'].reshape(m - 78200, 1)

    X = np.vstack([x_train, x_test]).reshape(m, 1024)
    Y = np.vstack([y_train, y_test]).reshape(m, 1)

    return X, Y

def shuffle(X, Y):
    if verbose:
        print('Shuffling data...')
    from sklearn.utils import shuffle
    X, Y = shuffle(X, Y)
    return X, Y

def plot(X, Y, n = 0):
    print('The image is character: ', characters[Y[n][0] - 1])
    plt.imshow(X[n, :].reshape(32, 32), cmap = 'Greys')
    plt.show()

def scale(X, factor = 255):
    if verbose:
        print('Scaling Features...')
    return X*(1/255)

def split(x, y, ratio = 0.2):
    if verbose:
        print('Splitting dataset...')
    from sklearn.model_selection import train_test_split
    return train_test_split(x, y, test_size = ratio)

def main():
    X, Y = load_data()
    X, Y = shuffle(X, Y)
    X = scale(X)
    x_train, x_test, y_train, y_test = split(X, Y)

    m_train = y_train.shape[0]
    m_test =y_test.shape[0]
   # plot(X,Y,1)
    

    #print('\nTotal Training Set size: ', m_train)
    #print('Total Test Set size: ', m_test)
    return x_train, x_test, y_train, y_test

x_train_full, x_test_full, y_train_full, y_test_full = main()
img_rows, img_cols = 32,32
#print(y_test)

x_train = x_train_full[0:20000]
y_train = y_train_full[0:20000] 
x_test = x_train_full[20000:28000]
y_test = y_train_full[20000:28000] 
print(x_test.shape)
print(y_test.shape)
print("type of x_test",type(x_test))
y_train = y_train.ravel()
y_test = y_test.ravel()


Loading dataset...
Shuffling data...
Scaling Features...
Splitting dataset...
(8000, 1024)
(8000, 1)
type of x_test <class 'numpy.ndarray'>


In [3]:
y_final=np.zeros((len(y_test_full),3))
y_final.shape

(18400, 3)

In [4]:
############################## H O G on full dataset ################################
"""
retuns, 
x_train_hog(73600,128)
x_test_hog(18400,128)
"""
x_train_hog=[]
x_test_hog=[]
for i in range(len(x_train_full)):
    x_train_feat1 = hog(np.reshape(x_train_full[i],(32,32)), orientations=8, pixels_per_cell=(8,8),
                        cells_per_block=(4, 4))
    x_train_hog.append(x_train_feat1)

for i in range(len(x_test_full)):
    x_test_feat1 = hog(np.reshape(x_test_full[i],(32,32)), orientations=8, pixels_per_cell=(8,8),
                        cells_per_block=(4, 4))
    x_test_hog.append(x_test_feat1)
    
x_train_hog=np.asarray(x_train_hog) 
x_test_hog=np.asarray(x_test_hog)    
print((x_train_hog).shape)
print((x_test_hog).shape)

C:\Users\Ashish\Anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


(73600, 128)
(18400, 128)


In [5]:
################################## K N N on full dataset #######################################
"""
KNN is applied after reducing the dimention with PCA and HOG,
for applying pca use "x_train_pca" and "y_train_pca". 
Test accuracy with PCA's dimention is 5.5%
for applying HOG use "x_train_feat" and "y_train_feat"
Test accuracy with HOG's dimention is 85.5%
"""
knn = KNeighborsClassifier(n_neighbors=5,n_jobs=-1,metric='braycurtis')
knn.fit(x_train_hog, y_train_full)
# get the model accuracy
model_score = knn.score(x_test_hog, y_test_full)
print(model_score)
y_final[:,1]=knn.predict(x_test_hog)
# 94.44% for n_neighbours=3
# 96.35% for n_neighbour =5
# save trained model
#joblib.dump(knn, 'knn_model.pkl')





# some time later...

# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))

C:\Users\Ashish\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


0.9635869565217391


In [10]:
import pickle
pickle.dump(knn, open('knn_model.sav', 'wb'))

In [12]:
####################### Random Forest ############################
from sklearn.ensemble import RandomForestClassifier
np.random.seed(0)

clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto',
             min_impurity_split=None,
            min_samples_leaf=1, 
            min_weight_fraction_leaf=0.0,  n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False,
                            max_leaf_nodes= 3000, min_impurity_decrease= 0.0, min_samples_split= 3, n_estimators=
                             160)

#clf.fit(x_train,y_train)      #on noraml image          
#clf.fit(x_train_hog,y_train)       # on HOG output
clf.fit(x_train_hog,y_train_full)   #on full dataset (hog)

C:\Users\Ashish\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=3000,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=3,
            min_weight_fraction_leaf=0.0, n_estimators=160, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [13]:
######################################### HOG on random forest full dataset #####################################
y_pred=clf.predict(x_test_hog)
sum1=0
for i in range (len(y_test_full)):
    if(y_test_full[i]==y_pred[i]):
        sum1=sum1+1
print('Test accuracy',sum1/len(y_test_full))

y_pred_train=clf.predict(x_train_hog)
sum1=0
for i in range (len(y_train_full)):
    if(y_train_full[i]==y_pred_train[i]):
        sum1=sum1+1
print('Train accuracy',sum1/len(y_train_full))

y_final[:,0]=y_pred

#88.8   1200
#89.175  #1400
#89.51   #1600
#89.975   #1900
#90.15      #2100  
#90.95    #3000              on 20,000 traning example  and 1100 leaves
#90.915   #2800

#Test accuracy 0.9248369565217391     on 3000 leaves and 160 estimator on full dataset
#Train accuracy 0.9885326086956522

Test accuracy 0.9248369565217391
Train accuracy 0.9885326086956522


In [14]:
pickle.dump(clf, open('random_forest_model.sav', 'wb'))

In [15]:
########################################## HOG + SVM #########################################################
from sklearn import svm
from sklearn.svm import LinearSVC

np.set_printoptions(threshold=np.inf)
#Assumed you have, X (predictor) and Y (target) for training data set and x_test(predictor) of test_dataset
# Create SVM classification object 
model = svm.SVC(kernel='linear',gamma=0.00001,C=90) #gamma is for rbf/polynomial kernal
#model=LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=50, multi_class='ovr', fit_intercept=True, intercept_scaling=10, class_weight=None, verbose=0, random_state=None, max_iter=1000)
# there is various option associated with it, like changing kernel, gamma and C value. Will discuss more # about it in next section.Train the model using the training sets and check score
model.fit(x_train_hog, y_train_full)
w=model.score(x_test_hog, y_test_full)
#Predict Output
predicted= model.predict(x_test_hog)
print(w)
y_final[:,2]=predicted
#with C=1 in svm we got 69.79% accuracy
#with C=90 in svm we got 94.07% accuracy (on full dataset)

C:\Users\Ashish\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.940054347826087


In [16]:
pickle.dump(model, open('hog_model.sav', 'wb'))

In [ ]:
"""
y_final[:,0]=random forest
y_final[:,1]=knn
y_final[:,2]=svm
"""
import pandas as pd
import numpy as np

df = pd.DataFrame(y_final)
a=df.mode(axis=1)
y_mode=np.zeros((len(a),1))
for i in range(len(a)):
    y_mode[i]=a[0][i]
#print(y_final)
sum1=0
print("y_final shape",y_final.shape)
print("y_mode shape",y_mode.shape)
print('Train accuracy'+str(np.sum(y_test_full==y_mode)/18400))



# test accuracy 96.0815%

In [19]:
"""
To load model

loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)

"""
y_final[:,0].shape

(18400,)

In [20]:
with open('y_random_forest.txt', 'w') as f:
    f.write(str(y_final[:,0]))  # Python 3.x

with open('y_knn.txt', 'w') as f:
    f.write(str(y_final[:,1]))  # Python 3.x

with open('y_svm.txt', 'w') as f:
    f.write(str(y_final[:,2]))  # Python 3.x